## Pivot points & EMA cross Strategy (custom v7)

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import pandas_ta as ta

In [2]:
import os
from pathlib import Path
notebook_path = os.getcwd()
algo_dir = Path(notebook_path).parent.parent.parent
csv_file = str(algo_dir) + '/vn-stock-data/VN30ps/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

In [3]:
FAST_EMA_LENGTH = 5
LOW_EMA_LENGTH = 26
def prepareData(htd):
    _1_d_df = htd.copy()
    _1_d_df['First_Open'] = _1_d_df['Open']
    _1_d_df['First_Close'] = _1_d_df['Close']
    _1_d_df['Second_Open'] = _1_d_df['Open']
    _1_d_df['Second_Close'] = _1_d_df['Close']
    _1_d_df = _1_d_df.resample("D").agg({
        'Open': 'first',
        'Close': 'last',
        'High': 'max',
        'Low': 'min',
        'Volume': 'sum',
        'First_Open': cal_first_open,
        'First_Close': cal_first_close,
        'Second_Open': cal_second_open,
        'Second_Close': cal_second_close
    })
    _1_d_df.dropna(inplace=True)
    _1_d_df = cal_pivots(_1_d_df)
    _1_d_df = _1_d_df[['P', 'R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'First_Open',
                       'First_Close', 'Second_Open', 'Second_Close']]
    _1_d_df.dropna(inplace=True)

    htd = htd.assign(time_d=pd.PeriodIndex(htd.index, freq='1D').to_timestamp())
    htd = pd.merge(htd, _1_d_df, left_on="time_d", right_index=True, how="left")
    htd["current"] = htd.index + pd.DateOffset(minutes=5)
    htd['prev_Close'] = htd['Close'].shift(1)
    htd["ema_f"] = ta.ema(htd["Close"], length=FAST_EMA_LENGTH)
    htd["ema_f_shift"] = htd["ema_f"].shift(1)
    htd["ema_l"] = ta.ema(htd["Close"], length=LOW_EMA_LENGTH)
    htd["ema_l_shift"] = htd["ema_l"].shift(1)
    htd.dropna(inplace=True)
    return htd


def cal_first_open(tick):
    tick = tick[100 * tick.index.hour + tick.index.minute == 900]
    return tick


def cal_first_close(tick):
    tick = tick[100 * tick.index.hour + tick.index.minute == 900]
    return tick


def cal_second_open(tick):
    tick = tick[100 * tick.index.hour + tick.index.minute == 905]
    return tick


def cal_second_close(tick):
    tick = tick[100 * tick.index.hour + tick.index.minute == 905]
    return tick


def cal_pivot(row):
    pivot = (row['High_s'] + row['Low_s'] + row['Close_s']) / 3
    return pivot


def cal_r1(row):
    result = 2 * row['P'] - row['Low_s']
    return result


def cal_r2(row):
    result = row['P'] + row['High_s'] - row['Low_s']
    return result


def cal_r3(row):
    # result = row['P'] + 2 * (row['High_s'] - row['Low_s'])    # Classic
    result = row['P'] * 2 + row['High_s'] - 2 * row['Low_s']
    return result


def cal_r4(row):
    # result = row['P'] + 3 * (row['High_s'] - row['Low_s'])    # Classic
    result = row['P'] * 3 + row['High_s'] - 3 * row['Low_s']
    return result


def cal_r5(row):
    # result = row['P'] + 4 * (row['High_s'] - row['Low_s'])      # Classic
    result = row['P'] * 4 + row['High_s'] - 4 * row['Low_s']
    return result


def cal_r6(row):
    # result = row['P'] + 5 * (row['High_s'] - row['Low_s'])      # Classic
    result = row['P'] * 5 + row['High_s'] - 5 * row['Low_s']
    return result


def cal_s1(row):
    result = 2 * row['P'] - row['High_s']
    return result


def cal_s2(row):
    result = row['P'] - (row['High_s'] - row['Low_s'])
    return result


def cal_s3(row):
    # result = row['P'] - 2 * (row['High_s'] - row['Low_s'])  # Classic
    result = row['P'] * 2 - (2 * row['High_s'] - row['Low_s'])
    return result


def cal_s4(row):
    # result = row['P'] - 3 * (row['High_s'] - row['Low_s'])  # Classic
    result = row['P'] * 3 - (3 * row['High_s'] - row['Low_s'])
    return result


def cal_s5(row):
    # result = row['P'] - 4 * (row['High_s'] - row['Low_s'])  # Classic
    result = row['P'] * 4 - (4 * row['High_s'] - row['Low_s'])
    return result


def cal_s6(row):
    # result = row['P'] - 5 * (row['High_s'] - row['Low_s'])  # Classic
    result = row['P'] * 5 - (5 * row['High_s'] - row['Low_s'])
    return result


def cal_pivots(_1_d_df):
    _1_d_df['High_s'] = _1_d_df['High'].shift(1)
    _1_d_df['Low_s'] = _1_d_df['Low'].shift(1)
    _1_d_df['Close_s'] = _1_d_df['Close'].shift(1)
    _1_d_df['Height_s'] = _1_d_df['High_s'] - _1_d_df['Low_s']
    _1_d_df['Volume_s'] = _1_d_df['Volume'].shift(1)

    _1_d_df['P'] = _1_d_df.apply(
        lambda row: cal_pivot(row), axis=1)
    _1_d_df['R1'] = _1_d_df.apply(
        lambda row: cal_r1(row), axis=1)
    _1_d_df['R2'] = _1_d_df.apply(
        lambda row: cal_r2(row), axis=1)
    _1_d_df['R3'] = _1_d_df.apply(
        lambda row: cal_r3(row), axis=1)
    _1_d_df['R4'] = _1_d_df.apply(
        lambda row: cal_r4(row), axis=1)
    _1_d_df['R5'] = _1_d_df.apply(
        lambda row: cal_r5(row), axis=1)
    _1_d_df['R6'] = _1_d_df.apply(
        lambda row: cal_r6(row), axis=1)
    _1_d_df['S1'] = _1_d_df.apply(
        lambda row: cal_s1(row), axis=1)
    _1_d_df['S2'] = _1_d_df.apply(
        lambda row: cal_s2(row), axis=1)
    _1_d_df['S3'] = _1_d_df.apply(
        lambda row: cal_s3(row), axis=1)
    _1_d_df['S4'] = _1_d_df.apply(
        lambda row: cal_s4(row), axis=1)
    _1_d_df['S5'] = _1_d_df.apply(
        lambda row: cal_s5(row), axis=1)
    _1_d_df['S6'] = _1_d_df.apply(
        lambda row: cal_s6(row), axis=1)
    return _1_d_df

In [4]:
data = dataset.copy()
prepared_data = prepareData(data)
prepared_data

,Open,High,Low,Close,Volume,time_d,P,R1,R2,R3,...,First_Open,First_Close,Second_Open,Second_Close,current,prev_Close,ema_f,ema_f_shift,ema_l,ema_l_shift
Date,,,,,,,,,,,,,,,,,,,,,
2018-08-14 09:00:00,955.5,955.5,954.7,954.9,1103,2018-08-14,950.366667,958.433333,962.666667,970.733333,...,955.5,954.9,954.8,955.0,2018-08-14 09:05:00,954.2,953.746859,953.170288,949.751340,949.339447
2018-08-14 09:05:00,954.8,955.0,954.3,955.0,530,2018-08-14,950.366667,958.433333,962.666667,970.733333,...,955.5,954.9,954.8,955.0,2018-08-14 09:10:00,954.9,954.164573,953.746859,950.140130,949.751340
2018-08-14 09:10:00,955.0,955.1,954.7,955.0,509,2018-08-14,950.366667,958.433333,962.666667,970.733333,...,955.5,954.9,954.8,955.0,2018-08-14 09:15:00,955.0,954.443048,954.164573,950.500120,950.140130
2018-08-14 09:15:00,955.0,957.0,955.0,956.9,1758,2018-08-14,950.366667,958.433333,962.666667,970.733333,...,955.5,954.9,954.8,955.0,2018-08-14 09:20:00,955.0,955.262032,954.443048,950.974185,950.500120
2018-08-14 09:20:00,956.5,956.6,955.6,955.8,1230,2018-08-14,950.366667,958.433333,962.666667,970.733333,...,955.5,954.9,954.8,955.0,2018-08-14 09:25:00,956.9,955.441355,955.262032,951.331653,950.974185
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-19 14:15:00,1301.2,1301.6,1298.3,1299.5,9058,2024-07-19,1296.800000,1305.700000,1313.900000,1322.800000,...,1304.6,1304.4,1304.4,1304.4,2024-07-19 14:20:00,1301.4,1300.456499,1300.934748,1302.015587,1302.216834
2024-07-19 14:20:00,1299.1,1301.8,1295.3,1301.6,12762,2024-07-19,1296.800000,1305.700000,1313.900000,1322.800000,...,1304.6,1304.4,1304.4,1304.4,2024-07-19 14:25:00,1299.5,1300.837666,1300.456499,1301.984803,1302.015587
2024-07-19 14:25:00,1302.4,1302.6,1300.1,1302.2,8096,2024-07-19,1296.800000,1305.700000,1313.900000,1322.800000,...,1304.6,1304.4,1304.4,1304.4,2024-07-19 14:30:00,1301.6,1301.291777,1300.837666,1302.000744,1301.984803


## Strategy

In [5]:
def trade_simulation(prepared_data):
    _trades = pd.DataFrame(columns=("EntryTime", "EntryPrice", "ExitTime", "ExitPrice", "Type", "Profit"))
    has_open_deal = False
    type_open_deal = ''
    # Stoploss at 20 pips
    sl_step = 3
    # Takeprofit at 60 pips(R/R = 1/3)
    tp_step = 9
    for i, row in prepared_data.iterrows():
            if has_open_deal:
                if type_open_deal == 'short':
                    if row['signal'] == 'close':
                        # close open deal at 2:25PM (dataframe)
                        profit = entry_price - row['Close']
                        if profit < -sl_step:
                            profit = -sl_step
                        exit_price = row['Close']
                        exit_time = row.name
                        has_open_deal = False
                        type_open_deal = ''
                        _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Short", profit]
                    # Stoploss
                    elif row['High'] > entry_price + sl_step:
                        profit = -sl_step
                        exit_price = entry_price + sl_step
                        exit_time = row.name
                        _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Short", profit]
                        has_open_deal = False
                        type_open_deal = ''
                    else:
                        """ Close short deal"""
                        body = abs(row['Close'] - row['Open'])
                        lower_shadow = min(row['Close'], row['Open']) - row['Low']
                        is_long_lower_shadow = True if lower_shadow > 2 else False
                        if body > 0.7 and is_long_lower_shadow and lower_shadow > 3 * body:
                            profit = entry_price - row['Close']
                            if profit < -sl_step:
                                profit = -sl_step
                            exit_price = row['Close']
                            exit_time = row.name
                            has_open_deal = False
                            type_open_deal = ''
                            _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Short", profit]
                    #     # Takeprofit
                    #     if row['Low'] < entry_price - tp_step:
                    #         profit = tp_step
                    #         exit_price = entry_price - tp_step
                    #         exit_time = row.name
                    #         _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Short", profit]
                    #         has_open_deal = False
                    #         type_open_deal = ''
                elif type_open_deal == 'long':
                    if row['signal'] == 'close':
                        # close open deal at 2:25PM (dataframe)
                        profit = row['Close'] - entry_price
                        if profit < -sl_step:
                            profit = -sl_step
                        exit_price = row['Close']
                        exit_time = row.name
                        has_open_deal = False
                        type_open_deal = ''
                        _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Long", profit]
                    # Stoploss
                    if row['Low'] < entry_price - sl_step:
                        profit = -sl_step
                        exit_price = entry_price - sl_step
                        exit_time = row.name
                        _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Long", profit]
                        has_open_deal = False
                        type_open_deal = ''
                    else:
                        """ Close long deal"""
                        body = abs(row['Close'] - row['Open'])
                        upper_shadow = row['High'] - max(row['Close'], row['Open'])
                        is_long_upper_shadow = True if upper_shadow > 2 else False
                        if body > 0.7 and is_long_upper_shadow and upper_shadow > 3 * body:
                            profit = row['Close'] - entry_price
                            if profit < -sl_step:
                                profit = -sl_step
                            exit_price = row['Close']
                            exit_time = row.name
                            has_open_deal = False
                            type_open_deal = ''
                            _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Long", profit]
                    #     # Takeprofit
                    #     if row['High'] > entry_price + tp_step:
                    #         profit = tp_step
                    #         exit_price = entry_price + tp_step
                    #         exit_time = row.name
                    #         _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Long", profit]
                    #         has_open_deal = False
                    #         type_open_deal = ''
                        #
            if not has_open_deal:
                if row['signal'] == 'short':
                    # Open short deal
                    entry_price = row['Close']
                    entry_time = row.name
                    has_open_deal = True
                    type_open_deal = 'short'
                elif row['signal'] == 'long':
                    # Open short deal
                    entry_price = row['Close']
                    entry_time = row.name
                    has_open_deal = True
                    type_open_deal = 'long'
    return _trades

In [6]:
def first_2_candlesticks_same_color(signal_data):
    """
        9:05AM has same candlestick's color with 9:00AM
    """
    first_is_green = True if signal_data['First_Close'] > signal_data['First_Open'] else False
    first_is_doji = True if signal_data['First_Close'] == signal_data['First_Open'] else False
    second_is_green = True if signal_data['Second_Close'] > signal_data['Second_Open'] else False
    second_is_doji = True if signal_data['Second_Close'] == signal_data['Second_Open'] else False

    if first_is_green != second_is_green and not second_is_doji and not first_is_doji:
        return False
    else:
        return True

In [7]:
def open_signal_v3(signal_data):
    """ Copying from custom V3"""
    signal = ''
    if signal_data['prev_Close'] < signal_data['P'] < signal_data['Close'] or signal_data['prev_Close'] < signal_data[
        'R1'] < signal_data['Close'] \
            or signal_data['prev_Close'] < signal_data['R2'] < signal_data['Close'] or signal_data['prev_Close'] < \
            signal_data['R3'] < signal_data['Close'] \
            or signal_data['prev_Close'] < signal_data['R4'] < signal_data['Close']:
        if (0.5 < signal_data['Close'] - signal_data['P'] < 3.5) \
                or (0.5 < signal_data['Close'] - signal_data['R1'] < 3.5) \
                or (0.5 < signal_data['Close'] - signal_data['R2'] < 3.5) \
                or (0.5 < signal_data['Close'] - signal_data['R3'] < 3.5) \
                or (0.5 < signal_data['Close'] - signal_data['R4'] < 3.5):
            signal = 'long'
    if signal_data['prev_Close'] > signal_data['P'] > signal_data['Close'] \
            or signal_data['prev_Close'] > signal_data['S1'] > signal_data['Close'] \
            or signal_data['prev_Close'] > signal_data['S2'] > signal_data['Close'] \
            or signal_data['prev_Close'] > signal_data['S3'] > signal_data['Close'] \
            or signal_data['prev_Close'] > signal_data['S4'] > signal_data['Close']:
        if (0.5 < signal_data['P'] - signal_data['Close'] < 3.5) \
                or (0.5 < signal_data['S1'] - signal_data['Close'] < 3.5) \
                or (0.5 < signal_data['S2'] - signal_data['Close'] < 3.5) \
                or (0.5 < signal_data['S3'] - signal_data['Close'] < 3.5) \
                or (0.5 < signal_data['S4'] - signal_data['Close'] < 3.5):
            signal = 'short'
    return signal
#
def open_signal_version_omega(signal_data):
    signal = ''
    if signal_data['ema_f_shift'] < signal_data['ema_l_shift'] and signal_data['ema_f'] > signal_data['ema_l']:
        signal = 'long'
    elif signal_data['ema_f_shift'] > signal_data['ema_l_shift'] and signal_data['ema_f'] < signal_data['ema_l']:
        signal = 'short'
    return signal

In [8]:
def cal_signal(row):
    signal = ''
    if 915 < 100*row.name.hour + row.name.minute < 1420:
        if row['same_color'] == True:
            signal = open_signal_v3(row)
        else:
            signal = open_signal_version_omega(row)
    elif 100*row.name.hour + row.name.minute == 1425:
        signal = 'close'
    return signal

In [9]:
prepared_data['same_color'] = prepared_data.apply(lambda r: first_2_candlesticks_same_color(r), axis=1)
prepared_data['signal'] = prepared_data.apply(lambda r: cal_signal(r), axis=1)
trade_result = trade_simulation(prepared_data)
trade_result.Profit.sum()

1773.2000000000007

In [10]:
total_trades = len(trade_result)
profit = trade_result.Profit.sum()
trading_fee = total_trades * 0.3
win_rate = len(trade_result[trade_result.Profit > 0]) / total_trades
display_data = {
    'trades' : total_trades,
    'profit' : profit,
    'win rate' : win_rate,
    'trading_fee' : trading_fee,
    'profit_after_fee' : profit - trading_fee
}
display_data

{'trades': 1824,
 'profit': 1773.2000000000007,
 'win rate': 0.3618421052631579,
 'trading_fee': 547.1999999999999,
 'profit_after_fee': 1226.000000000001}

In [11]:
trade_result.Profit.mean()

0.9721491228070179

In [12]:
# win rate
len(trade_result[trade_result.Profit > 0]) / (len(trade_result[trade_result.Profit < 0]) + len(trade_result[trade_result.Profit > 0]))

0.36423841059602646

In [13]:
trade_result

,EntryTime,EntryPrice,ExitTime,ExitPrice,Type,Profit
0,2018-08-14 10:45:00,953.0,2018-08-14 13:15:00,956.0,Short,-3.0
1,2018-08-15 11:20:00,956.5,2018-08-15 13:55:00,959.5,Short,-3.0
2,2018-08-15 14:15:00,951.5,2018-08-15 14:25:00,951.2,Short,0.3
3,2018-08-16 09:25:00,940.8,2018-08-16 14:00:00,943.8,Short,-3.0
4,2018-08-17 10:55:00,948.8,2018-08-17 11:05:00,951.8,Short,-3.0
...,...,...,...,...,...,...
1819,2024-07-15 14:15:00,1293.1,2024-07-15 14:25:00,1297.0,Short,-3.0
1820,2024-07-16 13:35:00,1304.2,2024-07-16 14:25:00,1301.8,Short,2.4
1821,2024-07-17 10:15:00,1302.4,2024-07-17 11:00:00,1305.4,Short,-3.0
1822,2024-07-17 13:50:00,1314.9,2024-07-17 14:20:00,1311.9,Long,-3.0


In [14]:
selected_month = trade_result[(trade_result.EntryTime > '2024-06-01 00:00:00') & (trade_result.EntryTime < '2024-06-30 15:00:00')]
selected_month.Profit.sum()

-22.600000000000136

In [15]:
from datetime import date
from datetime import timedelta
from datetime import datetime
last_expire_date = date.today() - timedelta(days=90)
last_expire_date = last_expire_date.strftime("%Y-%m-%d 00:00:00")
_last_3months = trade_result[trade_result.EntryTime > last_expire_date]
_last_3months.Profit.sum()

37.40000000000032

In [16]:
_last_3months

,EntryTime,EntryPrice,ExitTime,ExitPrice,Type,Profit
1761,2024-04-26 09:55:00,1226.8,2024-04-26 14:25:00,1232.1,Long,5.3
1762,2024-05-06 10:20:00,1259.2,2024-05-06 14:25:00,1274.0,Long,14.8
1763,2024-05-07 10:20:00,1267.8,2024-05-07 14:25:00,1276.3,Long,8.5
1764,2024-05-08 13:00:00,1268.1,2024-05-08 13:40:00,1271.1,Short,-3.0
1765,2024-05-08 14:00:00,1274.2,2024-05-08 14:20:00,1271.2,Long,-3.0
...,...,...,...,...,...,...
1819,2024-07-15 14:15:00,1293.1,2024-07-15 14:25:00,1297.0,Short,-3.0
1820,2024-07-16 13:35:00,1304.2,2024-07-16 14:25:00,1301.8,Short,2.4
1821,2024-07-17 10:15:00,1302.4,2024-07-17 11:00:00,1305.4,Short,-3.0
1822,2024-07-17 13:50:00,1314.9,2024-07-17 14:20:00,1311.9,Long,-3.0
